**Notebook 03 is a "run-once" setup**

- 📝 NOTEBOOK 3 - SETUP ONLY
- ✅ LLM client configured
- ✅ Prompt templates defined  
- ✅ Answer generator ready

No files saved - this notebook only needs to run once per session

# LLM Response Generation

**Why we're doing this:**
 Take retrieved document chunks and generate coherent answers using a language model.

**What we're doing:**

- Setting up first prototype - done
- Setting up the LLM client (Groq/Llama) - done
- Creating prompt templates for TRL questions - done
- Generating answers from retrieved context - done 

In [1]:
# PERMANENT WORKING IMPORT - USE THIS EVERYWHERE
import sys
import os
import importlib.util

def import_rag_components():
    """Import RAG components"""
    current_dir = os.getcwd()
    
    # Import retriever
    retriever_path = os.path.join(current_dir, 'rag_components', 'retriever.py')
    spec = importlib.util.spec_from_file_location("retriever", retriever_path)
    retriever_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(retriever_module)
    
    # Import query_interface  
    query_interface_path = os.path.join(current_dir, 'rag_components', 'query_interface.py')
    spec = importlib.util.spec_from_file_location("query_interface", query_interface_path)
    query_interface_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(query_interface_module)
    
    # Import answer_generator
    answer_generator_path = os.path.join(current_dir, 'rag_components', 'answer_generator.py')
    spec = importlib.util.spec_from_file_location("answer_generator", answer_generator_path)
    answer_generator_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(answer_generator_module)
    
    return (retriever_module.DocumentAwareRetriever, 
            query_interface_module.SimpleQueryInterface,
            answer_generator_module.RAGAnswerGenerator)

# Import the components
DocumentAwareRetriever, SimpleQueryInterface, RAGAnswerGenerator = import_rag_components()
print("🎉 COMPONENTS IMPORTED SUCCESSFULLY!")

# Continue with code
VECTOR_INDEX_PATH = "../../04_models/vector_index"
retriever = DocumentAwareRetriever(VECTOR_INDEX_PATH)
query_interface = SimpleQueryInterface(retriever)
answer_generator = RAGAnswerGenerator(query_interface)
print("✅ Generation pipeline ready!")

🎉 COMPONENTS IMPORTED SUCCESSFULLY!
✓ TF-IDF retriever loaded successfully
✓ Template-based RAG answer generator initialized
✅ Generation pipeline ready!


In [2]:
pip install groq

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
# CELL: LLM Client Setup
import os
from groq import Groq
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Groq client
def setup_groq_client():
    """Set up and return Groq client with error handling"""
    api_key = os.getenv('GROQ_API_KEY')
    
    if not api_key:
        raise ValueError("❌ GROQ_API_KEY not found in environment variables")
    
    client = Groq(api_key=api_key)
    print("✅ Groq client initialized successfully")
    return client

# Test the client
try:
    groq_client = setup_groq_client()
    print("🎉 LLM client ready for integration!")
except Exception as e:
    print(f"❌ Failed to initialize LLM client: {e}")

✅ Groq client initialized successfully
🎉 LLM client ready for integration!


In [4]:
# CELL: Test LLM Connection
# Why: Verify Groq API works and model responds correctly
# What: Send simple test query to confirm setup is functional
def test_llm_connection():
    try:
        response = groq_client.chat.completions.create(
            model="llama-3.1-8b-instant",  # Fast, free model for testing
            messages=[{"role": "user", "content": "Reply only with 'API connected'"}],
            max_tokens=10,
            temperature=0.1
        )
        print(f"✅ LLM Connected: {response.choices[0].message.content}")
        return True
    except Exception as e:
        print(f"❌ LLM Failed: {e}")
        return False

test_llm_connection()

✅ LLM Connected: API connected


True

In [5]:
# CELL: Integrate with Your Generator
def generate_with_llm(query, context):
    """Generate answer using Groq/Llama"""
    prompt = f"""
    Based on the following context, answer the user's question.
    
    Context: {context}
    
    Question: {query}
    
    Answer:
    """
    
    response = groq_client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=500,
        temperature=0.3
    )
    
    return response.choices[0].message.content

print("🚀 LLM integration code ready!")

🚀 LLM integration code ready!


In [6]:
# CELL: Universal Prompt Template
# Why: Single template that adapts to both regular and TRL queries automatically
# What: Smart template that detects when to include maturity analysis

UNIVERSAL_PROMPT_TEMPLATE = """
CONTEXT:
{context}

USER QUESTION:
{question}

ANALYSIS INSTRUCTIONS:
1. Provide a comprehensive answer based strictly on the context provided
2. Cite specific sources for each key point using [Source: filename]
3. If the context is insufficient, acknowledge what cannot be answered

{trl_section}

ADDITIONAL GUIDELINES:
- For technology maturity questions: assess development stage and transition evidence
- For trend questions: identify velocity, drivers, and key players  
- For forecasting: distinguish near-term vs long-term developments
- For descriptive questions: provide specific examples and entities

ANSWER:
"""

def build_smart_prompt(question, context):
    """Build adaptive prompt that includes TRL guidance only when needed"""
    
    # Detect if this is a technology maturity question
    maturity_keywords = ['trl', 'mature', 'transition', 'academy to application', 
                        'commercial', 'moving from academy', 'readiness', 'development stage']
    
    question_lower = question.lower()
    is_maturity_question = any(keyword in question_lower for keyword in maturity_keywords)
    
    # Include TRL section only for maturity questions
    if is_maturity_question:
        trl_section = """
TECHNOLOGY MATURITY ASSESSMENT:
- When discussing technology readiness, reference these stages:
  * Research Phase (TRL 1-4): Basic research, lab validation
  * Development Phase (TRL 5-6): Prototyping, testing  
  * Commercialization Phase (TRL 7-9): Deployment, scaling
- Assess current stage based on evidence in context
- Identify transition indicators and timelines
"""
    else:
        trl_section = ""
    
    prompt = UNIVERSAL_PROMPT_TEMPLATE.format(
        context=context,
        question=question,
        trl_section=trl_section
    )
    
    return prompt

# Test the universal template
def test_universal_prompt():
    """Test that the template adapts to different question types"""
    
    test_context = "Sample context about technology development..."
    
    # Test regular question
    regular_question = "Which startups work on AI for automotive?"
    regular_prompt = build_smart_prompt(regular_question, test_context)
    print("🔹 REGULAR QUESTION PROMPT:")
    print("Includes TRL section:", "TECHNOLOGY MATURITY ASSESSMENT" in regular_prompt)
    print("---")
    
    # Test TRL question  
    trl_question = "Which quantum computing research is moving from academy to application?"
    trl_prompt = build_smart_prompt(trl_question, test_context)
    print("🔹 TRL QUESTION PROMPT:")
    print("Includes TRL section:", "TECHNOLOGY MATURITY ASSESSMENT" in trl_prompt)
    
    return regular_prompt, trl_prompt

# Run test
regular_prompt, trl_prompt = test_universal_prompt()

print("\n✅ Universal prompt template ready!")
print("✅ Automatically includes TRL guidance for maturity questions")
print("✅ Single template for all query types")

🔹 REGULAR QUESTION PROMPT:
Includes TRL section: False
---
🔹 TRL QUESTION PROMPT:
Includes TRL section: True

✅ Universal prompt template ready!
✅ Automatically includes TRL guidance for maturity questions
✅ Single template for all query types


# Response Quality Setup

**Why we're doing this:** 
Ensure answers are relevant and properly cite sources.

**What we're doing:**

- Checking if the pipeline works and our LLM integration and prompt template can return something nice. 


In [11]:
# CELL: Test All User Queries with Dynamic Source Count & Startup Booster
# Why: Validate pipeline performance with intelligent source retrieval and startup boosting
# What: Run all 8 user queries with dynamic k-value and startup file enhancement

import json
import os
from datetime import datetime

def determine_source_count(question):
    """Dynamically determine how many sources to retrieve based on question type"""
    question_lower = question.lower()
    
    if any(keyword in question_lower for keyword in ['summarize', 'trends', 'overview', 'comprehensive']):
        return 5  # More sources for comprehensive questions
    elif any(keyword in question_lower for keyword in ['which', 'list', 'show me']):
        return 4  # Medium for listing questions
    elif any(keyword in question_lower for keyword in ['specific', 'exact', 'precise']):
        return 2  # Fewer for very specific questions
    else:
        return 3  # Default

def format_source_name(source_file):
    """Convert file names to human-readable format for better UX"""
    name_mapping = {
        # Automotive Papers
        'a_benchmark_framework_for_AL_models_in_automotive_aerodynamics.txt': 'Benchmark Framework for AI Models in Automotive Aerodynamics',
        'AL_agents_in_engineering_design_a_multiagent_framework_for_aesthetic_and_aerodynamic_car_design.txt': 'AI Agents in Engineering Design: Multiagent Framework for Car Design',
        'automating_automotive_software_development_a_synergy_of_generative_AL_and_formal_methods.txt': 'Automating Automotive Software Development: Generative AI and Formal Methods',
        'automotive-software-and-electronics-2030-full-report.txt': 'Automotive Software and Electronics 2030 Report',
        'drive_disfluency-rich_synthetic_dialog_data_generation_framework_for_intelligent_vehicle_environments.txt': 'DRIVE Framework: Synthetic Dialog Data for Intelligent Vehicles',
        'Embedded_acoustic_intelligence_for_automotive_systems.txt': 'Embedded Acoustic Intelligence for Automotive Systems',
        'enhanced_drift_aware_computer_vision_achitecture_for_autonomous_driving.txt': 'Enhanced Drift-Aware Computer Vision for Autonomous Driving',
        'Gen_AL_in_automotive_applications_challenges_and_opportunities_with_a_case_study_on_in-vehicle_experience.txt': 'Generative AI in Automotive: Applications and Challenges',
        'generative_AL_for_autonomous_driving_a_review.txt': 'Generative AI for Autonomous Driving: A Review',
        'leveraging_vision_language_models_for_visual_grounding_and_analysis_of_automative_UI.txt': 'Vision-Language Models for Automotive UI Analysis',
        
        # Tech Reports
        'bog_ai_value_2025.txt': 'Boston Consulting Group: AI Value Creation 2025',
        'mckinsey_tech_trends_2025.txt': 'McKinsey Technology Trends Outlook 2025',
        'wef_emerging_tech_2025.txt': 'World Economic Forum: Emerging Technologies 2025',
        'startups_processed.txt': 'Startup Innovation Database',
    }
    return name_mapping.get(source_file, source_file.replace('.txt', '').replace('_', ' ').title())

# Define user queries
USER_QUERIES = {
    1: "Which startups work on AI for automotive?",
    2: "Summarize the latest research on AI and autonomous driving.",
    3: "Summarize latest tech trends in development of AI agents",
    4: "Summarize the key pain points/use cases mentioned in these sources about automotive AI.",
    5: "Show me recent reports on technology trends.",
    6: "Which technologies are likely to mature next year?",
    7: "Which AI research topics are growing fastest?",
    8: "Which research topics in quantum computing are moving from academy to application?"
}

def test_complete_pipeline(question, query_id):
    """Test the full RAG pipeline with dynamic source count and startup boosting"""
    print(f"🧪 QUERY {query_id}: '{question}'")
    print("=" * 60)
    
    try:
        # Step 1: Determine optimal source count
        k = determine_source_count(question)
        print(f"1. 🔍 Retrieving documents (k={k})...")
        
        # Step 2: Retrieve documents
        retrieved_data = retriever.retrieve_with_sources(question, k=k)
        
        # 🚀 STARTUP BOOSTER: Enhance results for startup-related queries
        startup_boost_applied = False
        if any(keyword in question.lower() for keyword in ['startup', 'company', 'venture', 'business']):
            print("   🚀 Boosting startups file for this query...")
            # Get additional results focusing on startups
            startup_data = retriever.retrieve_with_sources(question + " startups companies", k=2)
            
            # Filter to only include startups file and avoid duplicates
            startup_items = []
            for item in startup_data:
                if 'startups_processed.txt' in item['source_file']:
                    # Check if this content is already in retrieved_data
                    is_duplicate = any(
                        item['content'] == existing['content'] 
                        for existing in retrieved_data
                    )
                    if not is_duplicate:
                        startup_items.append(item)
            
            # Add startup items to the beginning of results
            if startup_items:
                retrieved_data = startup_items + retrieved_data
                retrieved_data = retrieved_data[:k]  # Keep original k limit
                startup_boost_applied = True
                print(f"   ✅ Added {len(startup_items)} startup-specific results")
        
        print(f"   ✅ Found {len(retrieved_data)} relevant chunks")
        
        # Step 3: Format context with human-readable source names
        context = "\n\n".join([
            f"Source: {format_source_name(item['source_file'])} | Type: {item['doc_type']}\nContent: {item['content']}"
            for item in retrieved_data
        ])
        
        # Step 4: Build smart prompt
        print("2. 📝 Building prompt...")
        prompt = build_smart_prompt(question, context)
        
        # Step 5: Generate answer using LLM
        print("3. 🤖 Generating answer with LLM...")
        response = groq_client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=500,
            temperature=0.3
        )
        
        answer = response.choices[0].message.content
        
        # Step 6: Prepare results
        result = {
            'query_id': query_id,
            'question': question,
            'answer': answer,
            'sources': retrieved_data,
            'retrieved_chunks': len(retrieved_data),
            'source_count_used': k,
            'startup_boost_applied': startup_boost_applied,  # 🆕 Track if booster was used
            'timestamp': datetime.now().isoformat(),
            'model_used': 'llama-3.1-8b-instant'
        }
        
        # Display results
        print("4. 📊 RESULTS:")
        print(f"ANSWER: {answer}")
        print(f"SOURCES: {len(retrieved_data)} documents (k={k})")
        if startup_boost_applied:
            print("   🚀 Startup boost was applied to this query")
        for i, item in enumerate(retrieved_data):
            readable_name = format_source_name(item['source_file'])
            boost_indicator = "🚀 " if 'startups_processed.txt' in item['source_file'] and startup_boost_applied else ""
            print(f"   {i+1}. {boost_indicator}{readable_name} (Score: {item['similarity_score']:.3f})")
        
        print("✅ Query completed successfully!\n")
        return result
        
    except Exception as e:
        print(f"❌ Pipeline error: {e}")
        import traceback
        traceback.print_exc()
        return None

# Create output directory
output_dir = "../../07_testsdemo/test_outputs/demo_results"
os.makedirs(output_dir, exist_ok=True)

# Test all queries
print("🚀 TESTING ALL USER QUERIES WITH DYNAMIC SOURCE COUNT & STARTUP BOOSTER")
print("Note: Predictive section not yet integrated - focusing on RAG performance\n")

all_results = []
successful_queries = 0

for query_id, question in USER_QUERIES.items():
    result = test_complete_pipeline(question, query_id)
    if result:
        all_results.append(result)
        successful_queries += 1
        
        # Save individual query result
        individual_file = f"{output_dir}/user_query_{query_id}_{datetime.now().strftime('%Y%m%d_%H%M')}.json"
        with open(individual_file, 'w', encoding='utf-8') as f:
            json.dump(result, f, indent=2, ensure_ascii=False)

# Save consolidated results
if all_results:
    consolidated_file = f"{output_dir}/all_user_queries_with_startup_boost_{datetime.now().strftime('%Y%m%d_%H%M')}.json"
    with open(consolidated_file, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, indent=2, ensure_ascii=False)
    
    print("🎉 TESTING COMPLETE!")
    print(f"✅ Successful queries: {successful_queries}/{len(USER_QUERIES)}")
    print(f"📁 Individual results saved to: {output_dir}/")
    print(f"📊 Consolidated results: {consolidated_file}")
    
    # Summary with source count and boost info
    print("\n📈 QUERY PERFORMANCE SUMMARY:")
    for result in all_results:
        boost_info = " 🚀" if result['startup_boost_applied'] else ""
        print(f"  Q{result['query_id']}: k={result['source_count_used']}, {len(result['sources'])} sources{boost_info}, {len(result['answer'])} chars")
        
else:
    print("💥 No queries completed successfully")

print(f"\n📝 Note: Predictive model integration pending - currently testing RAG pipeline only")

🚀 TESTING ALL USER QUERIES WITH DYNAMIC SOURCE COUNT & STARTUP BOOSTER
Note: Predictive section not yet integrated - focusing on RAG performance

🧪 QUERY 1: 'Which startups work on AI for automotive?'
1. 🔍 Retrieving documents (k=4)...
   🚀 Boosting startups file for this query...
   ✅ Added 2 startup-specific results
   ✅ Found 4 relevant chunks
2. 📝 Building prompt...
3. 🤖 Generating answer with LLM...
4. 📊 RESULTS:
ANSWER: Based on the provided context, there is no direct information about startups working on AI for automotive applications. However, we can make some inferences and connections.

The research paper "Gen Ai In Automotive Applications Challenges And Opportunities With A Case Study On In-Vehicle Experience" [Source: Gen Ai In Automotive Applications Challenges And Opportunities With A Case Study On In-Vehicle Experience] discusses the impact of generative AI on the automotive industry, focusing on its applications, benefits, and potential challenges. This paper explores 